In [1]:
import Ativo as at
import FileManager as fman
import pickle
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
import TradesAnalyser as ta
import seaborn as sns

pd.set_option('display.max_rows', 167) # opcional, questão de visualização para debugar caso 1

In [2]:
fm = fman.FileManager()

In [3]:
with open('AtivoDiaList.pkl', 'rb') as filehandle:
    ativo_dia_list = pickle.load(filehandle)

In [4]:
an = ta.TradesAnalyser(ativo_dia_list)

In [5]:
# an.parameters.setFilterParameters(gap_threshold=0.1, F_high_threshold=9)
# an.parameters.setFilterParameters(prevol_threshold=15000000)
an.runFiltering()

In [ ]:
#an.parameters.setSimParameters(start_money = 10000,allocation=0.1)
#an.parameters.setAlgoParameters(short_after=0, exit_target=0.4, exit_stop=0.4)
#an.parameters.setAlgoParameters(short_after=0.2)

an.runSimulation()
# an.saveTrades('case1_trades.pkl')

# an.openTrades('case1_trades.pkl')

In [ ]:
#an.saveTrades('case4_trades.pkl')

In [ ]:
dfd = an.getFilteredDays() # DataFrame of filtered ativo-dias
dft = an.getTrades() # DataFrame of trades

##### Análise de algumas estratificações

In [ ]:
dftr = pd.merge(dft,dfd,how='inner',on=['name','date'])
dftr = dftr[['name','date','gap','factorF','profit','cum_profit']]
dftr

In [ ]:
bins = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2]
gap = np.clip( dftr['gap'], bins[0], bins[-1] )
dftr.groupby(pd.cut(gap, bins)).agg(
    winrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0)/(sum(x>=0) + sum(x<0)) ),
    lossrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0)/(sum(x>=0) + sum(x<0)) ),
    mean=pd.NamedAgg(column='profit',aggfunc=lambda x: x.mean()),
    mean_of_pos=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x>=0].mean()),
    mean_of_neg=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x<0].mean()),
    count=pd.NamedAgg(column='profit',aggfunc='count'),
    positives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0) ),
    negatives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0) )
)

In [ ]:
dftr.groupby(dftr['date'].dt.to_period('M')).agg(
    winrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0)/(sum(x>=0) + sum(x<0)) ),
    lossrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0)/(sum(x>=0) + sum(x<0)) ),
    mean=pd.NamedAgg(column='profit',aggfunc=lambda x: x.mean()),
    mean_of_pos=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x>=0].mean()),
    mean_of_neg=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x<0].mean()),
    count=pd.NamedAgg(column='profit',aggfunc='count'),
    positives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0) ),
    negatives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0) )
)

In [ ]:
an.printSimResults()

In [ ]:
an.plotHistMinsToTrade()

In [ ]:
an.plotEquityCurve(logy=False)

In [ ]:
an.getMaxDrawdown()

In [ ]:
an.getSimResults()

In [ ]:
an.runBootstrap(n_iter=50, replace=False)
an.printBootstrapResults()

In [ ]:
an.getBootstrapResults()

##### Códigos para rodar combinações de parâmetros e otimizar.

In [ ]:
# oficial que gostaríamos de explorar
an.runSimulationGroup(
    prevol_threshold = [800000],
    exit_target=[0.1, 0.2, 0.3, 0.4], 
    exit_stop=[0.1, 0.2, 0.3, 0.4], 
    short_after=[0, 0.1, 0.2, 0.3],
    gap_threshold=[0, 0.1, 0.2],
    F_high_threshold = [1,2,3,4,5]
)

In [ ]:
# an.saveGroupResults('erich01.pkl')
an.openGroupResults('erich01.pkl')

In [ ]:
# rodar para debugar
an.runSimulationGroup(
    prevol_threshold = [15000000],
    exit_target=[0.2, 0.3], 
    exit_stop=[0.3], 
    short_after=[0, 0.1],
    gap_threshold=[0.1],
    F_high_threshold = [1]
)

In [ ]:
an.results

##### Descrição simulações já rodadas

##### Testes temporários em geral